In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms, datasets
import foolbox as fb
from foolbox.attacks import LinfPGD

import os


# CIFAR 10

In [ ]:
def save_image(img, path):
    img = img.squeeze(0)  # remove batch dimension if present
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imsave(path, np.transpose(npimg, (1, 2, 0)))

# load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

# create directories for ImageFolder structure
output_dir = 'cifar10_adversarial_epsilon_02'
original_dir = os.path.join(output_dir, 'original')
adversarial_dir = os.path.join(output_dir, 'adversarial')

os.makedirs(original_dir, exist_ok=True)
os.makedirs(adversarial_dir, exist_ok=True)

# create subdirectories for each class
for class_idx in range(10):
    os.makedirs(os.path.join(original_dir, str(class_idx)), exist_ok=True)
    os.makedirs(os.path.join(adversarial_dir, str(class_idx)), exist_ok=True)

# define a simple model (using pretrained ResNet18)
model = torchvision.models.resnet18(pretrained=True)
model.eval()

# create Foolbox model
fmodel = fb.PyTorchModel(model, bounds=(-1, 1))

# define adversarial attack
attack = LinfPGD()
epsilons = [0.3] # TODO: play around with this for exaperimens 

# process and save each image
counter = 0 
for idx, (images, labels) in enumerate(testloader):
    if counter ==500: # TODO: remove this, at the moment is only saving ~500 images 
        break 
    label = labels.item()  #get the class label
    
    # save original image
    original_path = os.path.join(original_dir, str(label), f"{idx}.png")
    save_image(images[0], original_path)
    
    # apply attack to generate adversarial image
    adversarials = attack(fmodel, images, labels, epsilons=epsilons)
    adversarial_image = adversarials[0]

    # ensure adversarial_image is a tensor
    if isinstance(adversarial_image, list):
        adversarial_image = adversarial_image[0]
    if isinstance(adversarial_image, torch.Tensor):
        adversarial_image = adversarial_image.cpu()
    
    # save adversarial image
    adversarial_path = os.path.join(adversarial_dir, str(label), f"{idx}.png")
    save_image(adversarial_image, adversarial_path)

    # print progress
    if idx % 100 == 0:
        print(f"Processed {idx}/{len(testloader)} images")
    counter +=1

print(idx," images saved in ImageFolder format")

# IMAGENET

In [3]:
# Function to save images
def save_image(img, path):
    img = img.squeeze(0)  # remove batch dimension if present
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imsave(path, np.transpose(npimg, (1, 2, 0)))

# Load mini-ImageNet dataset
data_dir = 'imagenet_mini/val'
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
testset = datasets.ImageFolder(root=data_dir, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

# Output directories
output_dir = 'imagenet_mini_adversarial_epsilon_00' 
os.makedirs(output_dir, exist_ok=True)
# Create subdirectories for each class (WNIDs)
class_names = testset.classes  # This gives the WNIDs
for class_name in class_names:
    os.makedirs(os.path.join(output_dir, class_name), exist_ok=True)

# Define a simple model (using pretrained ResNet18)
model = torchvision.models.resnet18(pretrained=True)
model.eval()

# Create Foolbox model
fmodel = fb.PyTorchModel(model, bounds=(-1, 1))

# Define adversarial attack
attack = LinfPGD()
epsilons = [0.0]  # 0.0, 0.1, 0.3, 0.5 TODO: Play around with this for experiments

# Track processed classes
processed_classes = set()

# save  adversarial images
for idx, (images, labels) in enumerate(testloader):
    label = labels.item()  # Get the class label
    class_name = class_names[label]  # Map label to WNID

    # Skip if this class already has an adversarial image
    if class_name in processed_classes:
        continue
    
    adversarial_path = os.path.join(output_dir, class_name, f"{idx}.png")

    if epsilons[0] == 0.0:
        # Save original image if epsilon is 0.0
        save_image(images.cpu().squeeze(0), adversarial_path)
    else: 
        # Apply attack to generate adversarial image
        adversarials = attack(fmodel, images, labels, epsilons=epsilons)
        adversarial_image = adversarials[0]

        # Ensure adversarial_image is a tensor
        if isinstance(adversarial_image, list):
            adversarial_image = adversarial_image[0]
        if isinstance(adversarial_image, torch.Tensor):
            adversarial_image = adversarial_image.cpu()

        # Save adversarial image
        save_image(adversarial_image, adversarial_path)
    
    processed_classes.add(class_name)

    # Print progress
    if len(processed_classes) % 10 == 0:
        print(f"Processed {len(processed_classes)}/{len(class_names)} classes")

    # Stop if all classes have been processed
    if len(processed_classes) == len(class_names):
        break

print(f"{len(processed_classes)} adversarial images saved, one per class.")

Processed 10/1000 classes
Processed 20/1000 classes
Processed 30/1000 classes
Processed 40/1000 classes
Processed 50/1000 classes
Processed 60/1000 classes
Processed 70/1000 classes
Processed 80/1000 classes
Processed 90/1000 classes
Processed 100/1000 classes
Processed 110/1000 classes
Processed 120/1000 classes
Processed 130/1000 classes
Processed 140/1000 classes
Processed 150/1000 classes
Processed 160/1000 classes
Processed 170/1000 classes
Processed 180/1000 classes
Processed 190/1000 classes
Processed 200/1000 classes
Processed 210/1000 classes
Processed 220/1000 classes
Processed 230/1000 classes
Processed 240/1000 classes
Processed 250/1000 classes
Processed 260/1000 classes
Processed 270/1000 classes
Processed 280/1000 classes
Processed 290/1000 classes
Processed 300/1000 classes
Processed 310/1000 classes
Processed 320/1000 classes
Processed 330/1000 classes
Processed 340/1000 classes
Processed 350/1000 classes
Processed 360/1000 classes
Processed 370/1000 classes
Processed 